<a href="https://colab.research.google.com/github/TobiasEveN/AnsibleHealthAssessmentTask/blob/main/Assessmenttask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install --quiet --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build

# Build the Docs API service
docs_service = build('docs', 'v1')

In [32]:
"""
Core functions to parse markdown (headings, nested lists, checkboxes, assignee mentions)
and create a Google Doc via the Google Docs API.

See README for instructions.
"""

import re
import sys
import logging
from typing import List, Tuple

# logger setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

SAMPLE_MARKDOWN = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# Simple markdown parser to produce blocks:
# Block is a tuple (type, content, meta)
# types: 'title', 'h2', 'h3', 'para', 'ul', 'checkbox', 'hr', 'footer'
def parse_markdown(md: str):
    lines = md.splitlines()
    blocks = []
    list_stack = []
    i = 0
    while i < len(lines):
        line = lines[i].rstrip()
        if not line:
            i += 1
            continue
        # Horizontal rule
        if re.match(r'^-{3,}$', line):
            blocks.append(('hr', None, None))
            i += 1
            continue
        # Heading 1
        m = re.match(r'^# (.+)', line)
        if m:
            blocks.append(('title', m.group(1).strip(), None))
            i += 1
            continue
        # Heading 2
        m = re.match(r'^## (.+)', line)
        if m:
            blocks.append(('h2', m.group(1).strip(), None))
            i += 1
            continue
        # Heading 3
        m = re.match(r'^### (.+)', line)
        if m:
            blocks.append(('h3', m.group(1).strip(), None))
            i += 1
            continue
        # Checkbox line
        m = re.match(r'^\s*-\s*\[([ xX])\]\s*(.+)', line)
        if m:
            checked = (m.group(1).lower() == 'x')
            content = m.group(2).strip()
            blocks.append(('checkbox', content, {'checked': checked}))
            i += 1
            continue
        # Unordered list (dash or star)
        m = re.match(r'^(\s*)[*-]\s+(.+)', line)
        if m:
            indent = len(m.group(1))
            content = m.group(2).strip()
            # read nested items (consecutive list lines with >= indent)
            blocks.append(('ul', content, {'indent': indent}))
            i += 1
            continue
        # Inline footer lines "Meeting recorded by:"
        m = re.match(r'^(Meeting recorded by:|Duration:)\s*(.+)', line)
        if m:
            key = m.group(1).strip()
            value = m.group(2).strip()
            blocks.append(('footer', value, {'key': key}))
            i += 1
            continue
        # Paragraph fallback
        blocks.append(('para', line, None))
        i += 1
    return blocks

# Small helper to escape Google Docs special characters if needed
def escape_text(s: str) -> str:
    return s.replace('\r', '')

# Example usage in Colab: build requests array for Google Docs API
def build_docs_requests_from_blocks(blocks: List[Tuple]):
    full_text = ""
    ranges = []  # list of tuples (block_type, start_index, end_index, meta)
    for btype, content, meta in blocks:
        start = len(full_text)
        if btype == 'hr':
            txt = "\n-------------------------\n"
        elif btype == 'title':
            txt = content + "\n"
        elif btype in ('h2', 'h3'):
            txt = content + "\n"
        elif btype == 'checkbox':
            # Unicode checkbox characters for wide compatibility
            box = "☐ "
            txt = f"{box}{content}\n"
        elif btype == 'ul':
            indent = meta.get('indent', 0)
            level = indent // 2
            txt = ("  " * level) + "• " + content + "\n"
        elif btype == 'footer':
            txt = content + "\n"
        else:
            txt = content + "\n"
        full_text += txt
        end = len(full_text)
        ranges.append((btype, start, end, meta))
    # Build requests
    requests = []
    requests.append({'insertText': {'location': {'index': 1}, 'text': full_text}})
    # Apply paragraph styles based on ranges
    for btype, start, end, meta in ranges:
        # DocsAPI starting at 1 for insertText location
        start_idx = 1 + start
        end_idx = 1 + end
        if btype == 'title':
            requests.append({'updateParagraphStyle': {
                'range': {'startIndex': start_idx, 'endIndex': end_idx},
                'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                'fields': 'namedStyleType'
            }})
        elif btype == 'h2':
            requests.append({'updateParagraphStyle': {
                'range': {'startIndex': start_idx, 'endIndex': end_idx},
                'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                'fields': 'namedStyleType'
            }})
        elif btype == 'h3':
            requests.append({'updateParagraphStyle': {
                'range': {'startIndex': start_idx, 'endIndex': end_idx},
                'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                'fields': 'namedStyleType'
            }})
        elif btype == 'checkbox':
            content_slice = full_text[start:end]
            for m in re.finditer(r'@(\w+)', content_slice):
                a_start = start_idx + m.start()
                a_end = start_idx + m.end()
                requests.append({'updateTextStyle': {
                    'range': {'startIndex': a_start, 'endIndex': a_end},
                    'textStyle': {'bold': True},
                    'fields': 'bold'
                }})
        elif btype == 'ul':
            pass
        elif btype == 'footer':
            # Apply italic style for footer lines
            requests.append({'updateTextStyle': {
                'range': {'startIndex': start_idx, 'endIndex': end_idx},
                'textStyle': {'italic': True},
                'fields': 'italic'
            }})
    return requests

if __name__ == "__main__":
    blocks = parse_markdown(SAMPLE_MARKDOWN)
    requests = build_docs_requests_from_blocks(blocks)
    # Print summary for debugging
    print("Parsed blocks:")
    for b in blocks:
        print(b)
    print("\nGenerated requests (preview):")
    for r in requests[:10]:
        print(r)
    print("\nTotal requests:", len(requests))

Parsed blocks:
('title', 'Product Team Sync - May 15, 2023', None)
('h2', 'Attendees', None)
('ul', 'Sarah Chen (Product Lead)', {'indent': 0})
('ul', 'Mike Johnson (Engineering)', {'indent': 0})
('ul', 'Anna Smith (Design)', {'indent': 0})
('ul', 'David Park (QA)', {'indent': 0})
('h2', 'Agenda', None)
('h3', '1. Sprint Review', None)
('ul', 'Completed Features', {'indent': 0})
('ul', 'User authentication flow', {'indent': 2})
('ul', 'Dashboard redesign', {'indent': 2})
('ul', 'Performance optimization', {'indent': 2})
('ul', 'Reduced load time by 40%', {'indent': 4})
('ul', 'Implemented caching solution', {'indent': 4})
('ul', 'Pending Items', {'indent': 0})
('ul', 'Mobile responsive fixes', {'indent': 2})
('ul', 'Beta testing feedback integration', {'indent': 2})
('h3', '2. Current Challenges', None)
('ul', 'Resource constraints in QA team', {'indent': 0})
('ul', 'Third-party API integration delays', {'indent': 0})
('ul', 'User feedback on new UI', {'indent': 0})
('ul', 'Navigation 

In [33]:
# Parse your sample markdown
blocks = parse_markdown(SAMPLE_MARKDOWN)
requests = build_docs_requests_from_blocks(blocks)

# Create a new Google Doc
doc = docs_service.documents().create(body={"title": "Assessment Task"}).execute()
doc_id = doc["documentId"]

# Insert the text and formatting
docs_service.documents().batchUpdate(
    documentId=doc_id,
    body={"requests": requests}
).execute()

# Provide the Google Doc link
print("✅ Google Doc created!")
print(f"https://docs.google.com/document/d/{doc_id}/edit")

✅ Google Doc created!
https://docs.google.com/document/d/1TA23tv16zuifABtaqO51qZL3fvqUipy9WaPDXwHYFWo/edit
